In [1]:
!wget https://github.com/jcpeterson/percept2vec/blob/master/animals.zip?raw=true
!unzip animals.zip?raw=true
!wget https://github.com/jcpeterson/percept2vec/blob/master/turkResults_CogSci2016.mat?raw=true
!mv turkResults_CogSci2016.mat\?raw\=true hum.mat
import scipy.io
hum = scipy.io.loadmat('hum.mat')
humMat = hum['simMatrix']

--2020-08-05 05:44:12--  https://github.com/jcpeterson/percept2vec/blob/master/animals.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jcpeterson/percept2vec/raw/master/animals.zip [following]
--2020-08-05 05:44:12--  https://github.com/jcpeterson/percept2vec/raw/master/animals.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jcpeterson/percept2vec/master/animals.zip [following]
--2020-08-05 05:44:12--  https://raw.githubusercontent.com/jcpeterson/percept2vec/master/animals.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response.

In [2]:
import tensorflow as tf
vgg_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True, input_shape = (224, 224, 3))
v_model = tf.keras.Sequential()

for l in vgg_model.layers[:-1]:
  v_model.add(l)

v_model.summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_______________________________________________________

In [3]:
import glob
import re
import numpy as np
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
data = {}
name=[]
path = "/content/images/*.png"

for file in glob.glob(path):
    temp1=image.load_img(file)
    temp2=re.findall(r'\d+', file)
    name.append(int(temp2[0]))
    data.update({int(temp2[0]) : temp1})

print(len(name))

mapping = {}

for i in range(120):
  data[name[i]] = np.expand_dims(data[name[i]], axis=0)
  data[name[i]] = preprocess_input(data[name[i]])
  feature = v_model.predict(data[name[i]])
  mapping.update({name[i] : feature})
  mapping[name[i]] = np.reshape(mapping[name[i]],4096)

120


In [4]:
name.sort()

li = []

for i in range(120):
  li.append(mapping[name[i]])

F = np.asarray(li)
F = np.reshape(F, (120,4096))
print(F)

Mat = F.dot(F.transpose())
OrigSimMat = Mat
Mat = np.reshape(Mat, (14400))
print(Mat)

[[0.         1.583843   0.         ... 0.         0.47101805 0.        ]
 [0.7050716  0.         1.9185694  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.01230222]
 ...
 [0.         5.9127946  0.         ... 0.         0.         0.        ]
 [0.         0.         2.611578   ... 0.         0.         0.        ]
 [0.         1.7740846  0.         ... 0.         0.         0.555869  ]]
[7379.021   3241.4468  1205.5059  ... 1785.7755   408.06665 3472.9758 ]


In [17]:
humMat = np.reshape(humMat,(14400,))

cor_mat = np.corrcoef(Mat, humMat)
r = cor_mat[0,1]

r2 = r**2
print(r2)


0.44783041526507644


In [18]:
humanSimMat=humMat.reshape(120,120)

uniMat=[]               # For considering only the unique entries since it is a symmetric matrix
for i in range(120):
  for j in range(i,120):
    uniMat.append(humanSimMat[i,j])

uniMat=np.asarray(uniMat)

print(uniMat.shape)

(7260,)


In [12]:
inp = []               
for i in range(120):
  for j in range(i,120):
    inp.append(F[i]*F[j])

inp = np.asarray(inp)
print(inp.shape )

inp2 = []               
for i in range(120):
  for j in range(120):
    inp2.append(F[i]*F[j])

inp2 = np.asarray(inp2)
print(inp2.shape )



(7260, 4096)
(14400, 4096)


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

reg = Ridge(solver='sag', fit_intercept=False)

parameters = {'alpha': [10,100,1000,1e4, 50000, 1e5,1e6]}

search = GridSearchCV(reg, parameters, scoring='neg_mean_squared_error', cv=6)
search.fit(inp, uniMat)

GridSearchCV(cv=6, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=False,
                             max_iter=None, normalize=False, random_state=None,
                             solver='sag', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [10, 100, 1000, 10000.0, 50000, 100000.0,
                                   1000000.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [14]:
best_reg = search.best_estimator_
print(best_reg)

Ridge(alpha=10000.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='sag', tol=0.001)


In [23]:
from sklearn.model_selection import cross_validate
a=cross_validate(best_reg,inp2,humMat,scoring="r2",cv=6)

In [24]:
a['test_score'].mean()

0.8566448604447382

In [25]:
PredSimMat = best_reg.predict(inp2)

cor_mat = np.corrcoef(PredSimMat, humMat)
r = cor_mat[0,1]

r2 = r**2
print(r2)

0.8926651863102967
